In [1]:
import cptdl as dl 
import cptio as io 
import xcast as xc 
import datetime as dt
import xarray as xr
import cartopy.crs as ccrs
import warnings 
from dask.distributed import Client

In [2]:
predictor_names = ['SEAS5.PRCP', 'METEOFRANCE8.PRCP', 'GCFS2p1.PRCP', 'SPSv3p5.PRCP']

args = { 
  'fdate': dt.datetime(2022, 5, 1),
  'first_year': 1993, 
  'final_year': 2016, 
  'predictor_extent': {
    'east': 75,
    'west': -26, 
    'north': 42, 
    'south': -40
  }, 
  'predictand_extent': {
    'east': 75,
    'west': -26, 
    'north': 42, 
    'south': -40
  }, 
  'lead_low': 1.5,
  'lead_high': 4.5, 
  'target': 'Jun-Sep',
  'filetype': 'cptv10.tsv'
}

X = [ dl.download(dl.hindcasts[i], i+'.tsv', **args, verbose=True, use_dlauth=False) for i in predictor_names]
Y = dl.download(dl.observations['UCSB.PRCP'], 'UCSB.PRCP.tsv', **args, verbose=True, use_dlauth=False)
F = [ dl.download(dl.forecasts[i], i+'.tsv', **args, verbose=True, use_dlauth=False) for i in predictor_names]

URL: https://iridl.ldeo.columbia.edu/SOURCES/.EU/.Copernicus/.CDS/.C3S/.ECMWF/.SEAS5/.hindcast/.prcp/S/%280000%201%20May%201993-2016%29/VALUES/L/1.5/4.5/RANGEEDGES/%5BL%5D//keepgrids/average/Y/-40/42/RANGEEDGES/X/-26/75/RANGEEDGES/%5BM%5D/average/c%3A/1000/(mm%20m-1)/%3Ac/mul/c%3A/86400/(s%20day-1)/%3Ac/mul/c%3A/122//units/(days)/def/%3Ac/mul/-999/setmissing_value/%5BX/Y%5D%5BL/S/add%5D/cptv10.tsv

DOWNLOADING: [*************************] (2898 KB) 0:00:05.807428
URL: https://iridl.ldeo.columbia.edu/SOURCES/.EU/.Copernicus/.CDS/.C3S/.Meteo_France/.System8/.hindcast/.prcp/S/%280000%201%20May%201993-2016%29/VALUES/L/1.5/4.5/RANGEEDGES/%5BL%5D//keepgrids/average/Y/-40/42/RANGEEDGES/X/-26/75/RANGEEDGES/%5BM%5D/average/c%3A/1000/(mm%20m-1)/%3Ac/mul/c%3A/86400/(s%20day-1)/%3Ac/mul/c%3A/122//units/(days)/def/%3Ac/mul/-999/setmissing_value/%5BX/Y%5D%5BL/S/add%5D/cptv10.tsv

DOWNLOADING: [*************************] (2836 KB) 0:00:05.894082
URL: https://iridl.ldeo.columbia.edu/SOURCES/.EU/.Coper

In [3]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    X = [ xc.regrid(i.expand_dims({'M': [0]}).aprod, Y.coords['X'].values, Y.coords['Y'].values) for i in X ]
    F = [ xc.regrid(i.expand_dims({'M': [0]}).aprod, Y.coords['X'].values, Y.coords['Y'].values) for i in F ]
    X = xr.concat(X, 'M')
    F = xr.concat(F, 'M')
    X.to_netcdf('fewsnet_x.nc')
    F.to_netcdf('fewsnet_f.nc')
    Y.to_netcdf('fewsnet_y.nc')

In [4]:
ND=5
preprocessing='minmax'
crossvalidation_window=3
activation = 'lin'
hidden_layer_size=5

client = Client()
poelm_xval = []
elr_xval = []
i = 1
start = dt.datetime.now()
X, Y = xc.align_chunks(X, Y.prcp.expand_dims({'C': [0]}), 10, 10 ) 
for x_train, y_train, x_test, y_test in xc.CrossValidator(X, Y, window=crossvalidation_window):
    ohc = xc.RankedTerciles()
    ohc.fit(y_train)
    ohc_y_train = ohc.transform(y_train)

    poelm = xc.cPOELM(ND=ND, lat_chunks=10, lon_chunks=10, hidden_layer_size=hidden_layer_size, activation=activation, preprocessing=preprocessing)
    poelm.fit(x_train, ohc_y_train, rechunk=False)
    poelm_preds = poelm.predict_proba(x_test, rechunk=False)
    poelm_xval.append(poelm_preds.isel(T=crossvalidation_window // 2))

    elr = xc.cExtendedLogisticRegression(lat_chunks=10, lon_chunks=10 )
    elr.fit(x_train, y_train, rechunk=False)
    elr_preds = elr.predict_proba(x_test, rechunk=False)
    elr_xval.append(elr_preds.isel(T=crossvalidation_window // 2))
    print('Fit Window {}/24 - {}'.format(i, dt.datetime.now() - start), end='\r')
    i+= 1
poelm_hcst = xr.concat(poelm_xval, 'S').mean('ND')
elr_hcst = xr.concat(elm_xval, 'S').mean('ND')


/Users/kylehall/anaconda3/envs/main/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1584: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,
/Users/kylehall/anaconda3/envs/main/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1584: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,
/Users/kylehall/anaconda3/envs/main/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1584: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,
/Users/kylehall/anaconda3/envs/main/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1584: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,


/Users/kylehall/anaconda3/envs/main/lib/python3.10/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/Users/kylehall/anaconda3/envs/main/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1584: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,
/Users/kylehall/anaconda3/envs/main/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1584: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,
/Users/kylehall/anaconda3/envs/main/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1584: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,
/Users/kylehall/anaconda3/envs/main/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1584: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,
/Users/k

2022-07-11 15:22:05,738 - distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
2022-07-11 15:22:13,078 - distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
2022-07-11 15:22:21,385 - distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
2022-07-11 15:23:13,645 - distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
2022-07-11 15:23:51,735 - distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
2022-07-11 15:23:58,029 - distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
2022-07-11 15:24:10,007 - distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
2022-07-11 15:24:23,877 - distributed.utils_perf - WARNING - full garbage collections took

/Users/kylehall/anaconda3/envs/main/lib/python3.10/site-packages/statsmodels/genmod/families/links.py:187: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)


2022-07-11 16:10:02,122 - tornado.application - ERROR - Uncaught exception GET /status/ws (::1)
HTTPServerRequest(protocol='http', host='localhost:8787', method='GET', uri='/status/ws', version='HTTP/1.1', remote_ip='::1')
Traceback (most recent call last):
  File "/Users/kylehall/anaconda3/envs/main/lib/python3.10/site-packages/tornado/websocket.py", line 954, in _accept_connection
    open_result = handler.open(*handler.open_args, **handler.open_kwargs)
  File "/Users/kylehall/anaconda3/envs/main/lib/python3.10/site-packages/tornado/web.py", line 3173, in wrapper
    return method(self, *args, **kwargs)
  File "/Users/kylehall/anaconda3/envs/main/lib/python3.10/site-packages/bokeh/server/views/ws.py", line 149, in open
    raise ProtocolError("Token is expired.")
bokeh.protocol.exceptions.ProtocolError: Token is expired.


/Users/kylehall/anaconda3/envs/main/lib/python3.10/site-packages/statsmodels/genmod/families/links.py:187: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)


In [6]:
elr_hcst = xr.concat(elr_xval, 'S').mean('ND')


In [10]:
poelm_hcst = xr.concat(poelm_xval, 'S')

In [13]:
poelm_hcst.isel(ND=3).values

2022-07-11 20:26:56,459 - distributed.worker - WARNING - Compute Failed
Key:       ('broadcast_to-concatenate-getitem-45cdfce143bc7ef2350abb630b17eada', 4, 0, 1, 0)
Function:  execute_task
args:      ((<function getitem at 0x10f8109d0>, (<function broadcast_to at 0x10c60c790>, array([[[[nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan]],

        [[nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan]],

        [[nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan]],

        ...,

        [[ 1.,  1.,  1.],
         [ 1.,  1.,  1.],
         [ 1.,  1.,  1.],
         [ 1.,  1.,  1.],
         [ 1.,  1.,  1.]],

        [[ 1.,  1.,  1.],
         [ 1.,  1.,  1.],
         [ 1.,  1.,  1.],
         [ 1.,  1.,  1.],
         [ 1.,  1.,  1.]],

        [[ 1.,  1.,  1.],
         [ 1.,  1.,  1

         [ 1.,  1.,  1.],
         [ 1.,  1.,  1.],
         [ 1.,  1.,  1.],
         [ 1.,  1.,  1.]]],


       [[[nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [
kwargs:    {}
Exception: "ValueError('operands could not be broadcast together with remapped shapes [original->remapped]: (32,40,5,3)  and requested shape (1,32,40,5,5)')"

2022-07-11 20:26:56,645 - distributed.worker - WARNING - Compute Failed
Key:       ('broadcast_to-concatenate-getitem-45cdfce143bc7ef2350abb630b17eada', 4, 1, 2, 0)
Function:  execute_task
args:      ((<function getitem at 0x10f8109d0>, (<function broadcast_to at 0x10c60c790>, array([[[[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],


         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 
kwargs:    {}
Exception: "ValueError('operands could not be broadcast together with remapped shapes [original->remapped]: (32,40,5,3)  and requested shape (1,32,40,5,5)')"

2022-07-11 20:26:56,786 - distributed.worker - WARNING - Compute Failed
Key:       ('broadcast_to-concatenate-getitem-45cdfce143bc7ef2350abb630b17eada', 4, 10, 4, 0)
Function:  execute_task
args:      ((<function getitem at 0x10f8109d0>, (<function broadcast_to at 0x10c60c790>, array([[[[nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan]],

        [[nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan]],

        [[nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan]

        [[nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan]],

        [[nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan]]],


       [[[nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan]],

        [[nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan]],

        [[nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan]
kwargs:    {}
Exception: "ValueError('operands could not be broadcast together with remapped shapes [original->remapped]: (32,4,5,3)  and requested shape (1,32,4,5,5)')"

2022-07-11 20:26:56,863 - distributed.worker - WARNING - Compute Failed
Key:       ('broadcast_to-concatenate-getitem-45cdfce143bc7ef2350abb630b17eada', 4, 4, 8, 0)
Funct

        [[nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan]],

        [[nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan]],

        [[nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan]]],


       [[[nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [
kwargs:    {}
Exception: "ValueError('operands could not be broadcast together with remapped shapes [original->remapped]: (32,40,5,3)  and requested shape (1,32,40,5,5)')"

2022-07-11 20:26:57,014 - distributed.worker - WARNING - Compute Failed
Key:       ('broadcast_to-concatenate-getitem-45cdfce143bc7ef2350abb630b17eada', 4, 9, 9, 0)
Function:  execute_task
args:      ((<function getitem at 0x10de20940>, (<function broadcast_to at 0x10abe8790>, array([[[[nan, nan, nan],
         [

...

args:      ((<function getitem at 0x1160c8940>, (<function broadcast_to at 0x112f00790>, array([[[[nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan]],

        [[nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan]],

        [[nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan]],

        ...,

        [[nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan]],

        [[nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan]],

        [[nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan]]],


       [[[nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],

ValueError: operands could not be broadcast together with remapped shapes [original->remapped]: (32,40,5,3)  and requested shape (1,32,40,5,5)

[nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan]]],


       [[[nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [
kwargs:    {}
Exception: "ValueError('operands could not be broadcast together with remapped shapes [original->remapped]: (32,40,5,3)  and requested shape (1,32,40,5,5)')"

2022-07-11 20:26:57,497 - distributed.worker - WARNING - Compute Failed
Key:       ('broadcast_to-concatenate-getitem-45cdfce143bc7ef2350abb630b17eada', 15, 1, 6, 0)
Function:  execute_task
args:      ((<function getitem at 0x10f8109d0>, (<function broadcast_to at 0x10c60c790>, array([[[[nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan]],

        [[nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan]],

        [[nan, nan, nan],
         [nan, nan, nan],
         [nan, nan

Exception: "ValueError('operands could not be broadcast together with remapped shapes [original->remapped]: (8,40,5,3)  and requested shape (1,8,40,5,5)')"

2022-07-11 20:26:57,597 - distributed.worker - WARNING - Compute Failed
Key:       ('broadcast_to-concatenate-getitem-45cdfce143bc7ef2350abb630b17eada', 15, 3, 4, 0)
Function:  execute_task
args:      ((<function getitem at 0x1160c8940>, (<function broadcast_to at 0x112f00790>, array([[[[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        ...,

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1.,

args:      ((<function getitem at 0x10f8109d0>, (<function broadcast_to at 0x10c60c790>, array([[[[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        ...,

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]]],


       [[[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
       